In [1]:
from scripts import setup_environment

setup_environment()

In [2]:
import textwrap

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)


def get_system_prompt(template_name: str):
    # System message templates (priming)
    system_templates = {
        "step_by_step": "You are a meticulous problem-solver. Analyze silently and return ONLY the correct answer text.",
        "creative": "You excel at lateral thinking. Treat this as a riddle. Return ONLY the option text.",
        "elimination": "Eliminate wrong options internally. Return ONLY the text of the correct answer.",
        "metaphor": "Interpret keywords metaphorically. Return ONLY the correct answer text.",
        "confidence": "Score options internally. Return ONLY the highest-scoring answer text.",
        "perspective_shift": "Analyze through multiple perspectives silently. Return ONLY the answer text.",
        "common_sense": "Combine logic and creativity. Return ONLY the correct answer text.",
        "assumption_challenge": "Challenge hidden assumptions internally. Return ONLY the answer text.",
        "pattern_matching": "Find patterns silently. Return ONLY the correct answer text.",
        "intuitive": "Critique your intuition internally. Return ONLY the final answer text.",
    }

    system_prompt = system_templates[template_name]
    system_prompt = textwrap.dedent(system_prompt)

    system_prompt_template = SystemMessagePromptTemplate.from_template(
        system_prompt, id=template_name
    )
    return system_prompt_template


def get_user_prompt():
    prompt = """
    Question: {question}
    Choices:
    {choices}
    """

    prompt = textwrap.dedent(prompt)

    prompt_template = HumanMessagePromptTemplate.from_template(prompt)
    return prompt_template


def create_prompt_template(
    system_prompt_template_name: str = "step_by_step",
):
    system_prompt_template = get_system_prompt(system_prompt_template_name)
    user_prompt_template = get_user_prompt()
    chat_prompt_template = ChatPromptTemplate.from_messages(
        [system_prompt_template, user_prompt_template]
    )

    return chat_prompt_template

In [ ]:
from scripts.dataset import BrainteaserDataset

dataset = BrainteaserDataset("data")

: 

In [ ]:
from scripts.llm.lmm import LLM

llm = LLM("google/gemma-2-2b-it")

In [9]:
chat_prompt_template = create_prompt_template("step_by_step")

riddle_question = dataset.sp_train[0]
template_args = {
    "question": riddle_question.question,
    "choices": "\n".join(
        [
            f"Option {j + 1}: {choice}"
            for j, choice in enumerate(riddle_question.choice_list)
        ]
    ),
}


display(chat_prompt_template)
result = llm.generate(chat_prompt_template, template_args)

display(result)

ChatPromptTemplate(input_variables=['choices', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a meticulous problem-solver. Analyze silently and return ONLY the correct answer text.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['choices', 'question'], input_types={}, partial_variables={}, template='\nQuestion: {question}\nChoices:\n{choices}\n'), additional_kwargs={})])

'System: You are a meticulous problem-solver. Analyze silently and return ONLY the correct answer text.\nHuman: \nQuestion: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?\nChoices:\nOption 1: Some daughters get married and have their own family.\nOption 2: Each daughter shares the same brother.\nOption 3: Some brothers were not loved by family and moved away.\nOption 4: None of above.\nAnswer: Option 4: None of above.'

In [ ]:
# from scripts.executor import ModelExecutor

# executor = ModelExecutor(
#     models=[
#         "google/gemma-2-2b-it",
#         "google/gemma-2-9b-it",
#         "microsoft/Phi-3.5-mini-instruct",
#         "meta-llama/Llama-3.1-8B-Instruct",
#         "meta-llama/Llama-3.2-1B-Instruct",
#         "meta-llama/Llama-3.2-3B-Instruct",
#         "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
#         "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
#         "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
#     ],
#     dataset=dataset.sp_train,
# )

# results = executor.run(create_prompt_template)

In [4]:
# for model, model_results in results.items():
#     print(f"Model: {model}")
#     for result in model_results:
#         print(f"Prompt: {result.prompt()}")
#         print(f"Output: {result.model_response()}")
#         print("-" * 50)

Model: meta-llama/Llama-3.2-1B-Instruct
Prompt: [{'role': 'system', 'content': 'You are a helpful AI assistant. You will be given a question with multiple choice answers. Your task is to select the correct answer from the given choices.'}, {'role': 'user', 'content': '\nQuestion: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?\nChoices:\nOption 1: Some daughters get married and have their own family.\nOption 2: Each daughter shares the same brother.\nOption 3: Some brothers were not loved by family and moved away.\nOption 4: None of above.\nAnswer: Please return the answer in the format of the choice.\n'}]
Output: Option 2: Each daughter shares the same brother.
--------------------------------------------------
Prompt: [{'role': 'system', 'content': 'You are a helpful AI assistant. You will be given a question with multiple choice answers. Your task is to select the correct answer from the give